GETTING airports list

In [5]:
#from pyspark.sql import SparkSession
import pandas as pd 
from FlightRadar24.api import FlightRadar24API
import os 
#spark = SparkSession.builder.getOrCreate()
#from pyspark.sql.types import StructType, StructField, FloatType , StringType , DoubleType
def getzones():
    #filename = "/workspaces/ProjetPerso/Airflow-docker/dags/STATDATA/zones.csv"

    fr_api = FlightRadar24API()
    zones = fr_api.get_zones()    
    schema=['zone','tl_y','tl_x','br_y','br_x']
    df_zones=pd.DataFrame(columns=schema)

    for zone in zones: 
        dfsec=pd.DataFrame([[zone,float(zones[zone]['tl_y']),float(zones[zone]['tl_x']),float(zones[zone]['br_y']),float(zones[zone]['br_x'])]],columns=schema)
        df_zones=pd.concat([df_zones,dfsec], ignore_index=True)
    df_zones.to_csv('statdata/zones2.csv',index=False)
    return df_zones

getzones()


,zone,tl_y,tl_x,br_y,br_x
0,europe,72.57,-16.96,33.57,53.05
1,northamerica,75.00,-180.00,3.00,-52.00
2,southamerica,16.00,-96.00,-57.00,-31.00
3,oceania,19.62,88.40,-55.08,180.00
4,asia,79.98,40.91,12.48,179.77
5,africa,39.00,-29.00,-39.00,55.00
6,atlantic,52.62,-50.90,15.62,-4.75
7,maldives,10.72,63.10,-6.08,86.53
8,northatlantic,82.62,-84.53,59.02,4.45


In [11]:
from pyspark.sql import SparkSession
import pandas as pd 
from FlightRadar24.api import FlightRadar24API
spark = SparkSession.builder.getOrCreate()
from pyspark import SparkContext 
from pyspark.sql.types import StructType, StructField, FloatType , StringType , DoubleType
fr_api = FlightRadar24API()

# Créer un objet SparkContext
sc = SparkContext("local", "example")

schema = StructType([
      StructField("airport_name", StringType(), True),
      StructField("iata", StringType(), True),
      StructField("icao", StringType(), True),
      StructField("lat", FloatType(), True),
      StructField("long", FloatType(), True),
      StructField("country", StringType(), True),
      StructField("airport_zone", StringType(), True),
      StructField("alt", FloatType(), True)
])

dfairports=spark.createDataFrame([],schema)

airports = fr_api.get_airports()  

dfzones=spark.read.csv('zones.csv', header=True, inferSchema=True)

def process_airport(airport):
    for row in dfzones.rdd.collect():
        if (row['tl_y'] >= airport['lat']) &  (row['br_y'] <= airport['lat']) & (row['br_x'] >= airport['lon']) &  (row['tl_x'] <= airport['lon']):
            continent=row['zone']
            break 
    df2=spark.createDataFrame([(airport['name'],airport['iata'],airport['icao'],float(airport['lat']),float(airport['lon']),airport['country'],continent,float(airport['alt']))],schema)
    return df2
# Convert the airports list to an RDD
rdd_airports = sc.parallelize(airports)

# Use map() to apply process_airport() function to each airport in parallel
rdd_airports_processed = rdd_airports.map(process_airport)

# Union all processed airports into the final DataFrame
dfairports = rdd_airports_processed.reduce(lambda x, y: x.union(y))

# Show the final DataFrame
dfairports.show()
dfairports.toPandas().to_csv('airports.csv',index=False)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at /tmp/ipykernel_6948/1522537035.py:4 

Airlines list

In [4]:
airlines = fr_api.get_airlines()
display(airlines)

[{'Name': '21 Air', 'Code': '2I', 'ICAO': 'CSB'},
 {'Name': '25only Aviation', 'Code': '4Q', 'ICAO': 'ONY'},
 {'Name': '2Excel Aviation', 'Code': '', 'ICAO': 'BRO'},
 {'Name': '40-Mile Air', 'Code': 'Q5', 'ICAO': 'MLA'},
 {'Name': '748 Air Services', 'Code': 'FE', 'ICAO': 'IHO'},
 {'Name': '9 Air', 'Code': 'AQ', 'ICAO': 'JYH'},
 {'Name': 'Abakan Air', 'Code': 'S5', 'ICAO': 'NKP'},
 {'Name': 'ABS Jets', 'Code': '', 'ICAO': 'ABP'},
 {'Name': 'Abu Dhabi Aviation', 'Code': '', 'ICAO': 'BAR'},
 {'Name': 'ABX Air', 'Code': 'GB', 'ICAO': 'ABX'},
 {'Name': 'Acass Ireland', 'Code': '', 'ICAO': 'SON'},
 {'Name': 'Advanced Air', 'Code': 'AN', 'ICAO': 'WSN'},
 {'Name': 'Aegean Airlines', 'Code': 'A3', 'ICAO': 'AEE'},
 {'Name': 'Aer Lingus', 'Code': 'EI', 'ICAO': 'EIN'},
 {'Name': 'Aer Lingus UK', 'Code': 'EG', 'ICAO': 'EUK'},
 {'Name': 'AerCaribe', 'Code': 'JK', 'ICAO': 'ACL'},
 {'Name': 'Aero', 'Code': 'TD', 'ICAO': 'BLK'},
 {'Name': 'Aero', 'Code': 'N2', 'ICAO': 'NIG'},
 {'Name': 'Aero Asahi Cor

Flights lists

In [5]:
flights = fr_api.get_flights()
display(flights)

[<(GLID) WINDRAD - Altitude: 1538 - Ground Speed: 1 - Heading: 37>,
 <(GLID) WINDRAD - Altitude: 1669 - Ground Speed: 1 - Heading: 89>,
 <(GLID) WINDRAD - Altitude: 1610 - Ground Speed: 1 - Heading: 217>,
 <(GLEX) N/A - Altitude: 41000 - Ground Speed: 507 - Heading: 55>,
 <(GLID) WINDRAD - Altitude: 1791 - Ground Speed: 1 - Heading: 271>,
 <(A359) N517DZ - Altitude: 0 - Ground Speed: 0 - Heading: 253>,
 <(A35K) B-LXN - Altitude: 38000 - Ground Speed: 401 - Heading: 289>,
 <(B789) C-FVLQ - Altitude: 0 - Ground Speed: 14 - Heading: 104>,
 <(A359) N513DZ - Altitude: 0 - Ground Speed: 0 - Heading: 188>,
 <(B789) ZK-NZQ - Altitude: 40000 - Ground Speed: 457 - Heading: 215>,
 <(A359) 9V-SGC - Altitude: 41000 - Ground Speed: 548 - Heading: 129>,
 <(B77W) A7-BAM - Altitude: 34000 - Ground Speed: 470 - Heading: 219>,
 <(B77L) B-2042 - Altitude: 6950 - Ground Speed: 229 - Heading: 285>,
 <(B77W) ZK-OKN - Altitude: 36000 - Ground Speed: 500 - Heading: 216>,
 <(B789) G-ZBKR - Altitude: 39975 - Gro

Zone list

In [12]:
from pyspark.sql import SparkSession
import pandas as pd 
from FlightRadar24.api import FlightRadar24API
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

from pyspark.sql.types import StructType, StructField, FloatType , StringType , DoubleType
fr_api = FlightRadar24API()

zones = fr_api.get_zones()    

schema = StructType([
      StructField("zone", StringType(), True),
      StructField("tl_y", FloatType(), True),
      StructField("tl_x", FloatType(), True),
      StructField("br_y", FloatType(), True),
      StructField("br_x", FloatType(), True),
])

df_zones=spark.createDataFrame([],schema)
for zone in zones: 
    dfsec=spark.createDataFrame([(zone,float(zones[zone]['tl_y']),float(zones[zone]['tl_x']),float(zones[zone]['br_y']),float(zones[zone]['br_x']))],schema)
    df_zones=df_zones.union(dfsec)
df_zones.toPandas().to_csv('statdata/zones.csv',index=False)
sc.stop()

In [6]:
#from pyspark.sql import SparkSession
import pandas as pd 
from FlightRadar24.api import FlightRadar24API
#spark = SparkSession.builder.getOrCreate()
#from pyspark.sql.types import StructType, StructField, FloatType , StringType , DoubleType
fr_api = FlightRadar24API()

zones = fr_api.get_zones()    

"""schema = StructType([
      StructField("zone", StringType(), True),
      StructField("tl_y", FloatType(), True),
      StructField("tl_x", FloatType(), True),
      StructField("br_y", FloatType(), True),
      StructField("br_x", FloatType(), True),
])
df_zones=spark.createDataFrame([],schema)"""
schema=['zone','tl_y','tl_x','br_y','br_x']
df_zones=pd.DataFrame(columns=schema)

for zone in zones: 
    #dfsec=spark.createDataFrame([(zone,float(zones[zone]['tl_y']),float(zones[zone]['tl_x']),float(zones[zone]['br_y']),float(zones[zone]['br_x']))],schema)
    #df_zones=df_zones.union(dfsec)
    dfsec=pd.DataFrame([[zone,float(zones[zone]['tl_y']),float(zones[zone]['tl_x']),float(zones[zone]['br_y']),float(zones[zone]['br_x'])]],columns=schema)
    df_zones=pd.concat([df_zones,dfsec], ignore_index=True)
#df_zones.toPandas().to_csv('zones.csv',index=False)
df_zones.to_csv('zones.csv',index=False)
display(df_zones)

,zone,tl_y,tl_x,br_y,br_x
0,europe,72.57,-16.96,33.57,53.05
1,northamerica,75.00,-180.00,3.00,-52.00
2,southamerica,16.00,-96.00,-57.00,-31.00
3,oceania,19.62,88.40,-55.08,180.00
4,asia,79.98,40.91,12.48,179.77
5,africa,39.00,-29.00,-39.00,55.00
6,atlantic,52.62,-50.90,15.62,-4.75
7,maldives,10.72,63.10,-6.08,86.53
8,northatlantic,82.62,-84.53,59.02,4.45


In [2]:
flights = fr_api.get_flights()
display(flights)

NameError: name 'fr_api' is not defined

More info about a flight a revoiiir : 

In [1]:
for flight in fr_api.get_flights():
    details = fr_api.get_flight_details(flight.id)
    flight.set_flight_details(details)
    #print(f"ID {flight.id} latitude {flight.latitude} longitude {flight.longitude} callsign {flight.callsign}")
    #print(f"Flying (id {flight.id}) is from  {flight.origin_airport_name} position  {flight.origin_airport_latitude}, {flight.origin_airport_longitude} to {flight.destination_airport_name} position  {flight.destination_airport_latitude}, {flight.destination_airport_longitude}")
    #display(details)
    print(f" code {flight.aircraft_code} time {flight.time_details}")

NameError: name 'fr_api' is not defined

In [ ]:
params = fr_api.get_real_time_flight_tracker_config()
set_real_time_flight_tracker_config(**new_config)

NameError: name 'set_real_time_flight_tracker_config' is not defined

Table vol distance 

In [47]:
from geopy.distance import geodesic
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
#@udf(returnType=DoubleType())
def calculate_distance(lon1, lat1, lon2, lat2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

colonnesvols=['id_vol','originairport_country','destinationairport_country','airline_icao','airline_iata','id_avion','originairport_icao','originairport_iata','destinationairport_icao','longueur_trajet']
df1=pd.DataFrame(columns=colonnesvols)
for flight in fr_api.get_flights():
    details = fr_api.get_flight_details(flight.id)
    #print(details)
    
    #list.append(id_vol)
    #print(flight.id)
    if (type(details)==dict):
        flight.set_flight_details(details)
        id_vol=flight.id
        airline_icao=flight.airline_icao
        airline_iata=flight.airline_iata
        id_avion=flight.aircraft_code
        originairport_icao=flight.origin_airport_icao
        originairport_iata=flight.origin_airport_iata
        destinationairport_icao=flight.destination_airport_latitu
        destinationairport_country=flight.destination_airport_country_name
        originairport_country=flight.origin_airport_country_name
        if (flight.origin_airport_latitude != "N/A" and flight.destination_airport_latitude != "N/A" and flight.destination_airport_longitude != "N/A" and flight.origin_airport_longitude != "N/A"):
            longueur_trajet=calculate_distance(float(flight.origin_airport_longitude),float(flight.origin_airport_latitude),float(flight.destination_airport_longitude),float(flight.destination_airport_latitude))
        else: 
            longueur_trajet =0
        df2=pd.DataFrame([[id_vol,originairport_country,destinationairport_country,airline_icao,airline_iata,id_avion,originairport_icao,originairport_iata,destinationairport_icao,longueur_trajet]],columns=colonnesvols)
        df1=pd.concat([df1,df2], ignore_index=True)
df1.to_csv('firstable.csv',index=False)
display(df1)
print(len(df1))

,id_vol,originairport_country,destinationairport_country,airline_icao,airline_iata,id_avion,originairport_icao,originairport_iata,destinationairport_icao,longueur_trajet
0,300b874c,N/A,N/A,N/A,N/A,GLID,N/A,N/A,N/A,0
1,300b87c8,N/A,N/A,N/A,N/A,GLID,N/A,N/A,N/A,0
2,300e9af0,N/A,N/A,N/A,N/A,GLID,N/A,N/A,N/A,0
3,300f02d5,N/A,N/A,N/A,N/A,GLEX,N/A,N/A,N/A,0
4,30136573,N/A,N/A,N/A,N/A,GLID,N/A,N/A,N/A,0
...,...,...,...,...,...,...,...,...,...,...
385,3016912d,Austria,United Arab Emirates,FDB,FZ,B38M,LOWS,SZG,OMDB,4462.635132
386,301691a7,United States,Turks And Caicos Islands,JBU,B6,A320,KJFK,JFK,MBPV,2096.702874
387,301691ab,Taiwan,Malaysia,BTK,OD,B38M,RCTP,TPE,WMKK,3238.616776
388,301691fa,United States,Aruba,AAL,AA,A321,KCLT,CLT,TNCA,2746.22813


390


Pour chque continent , compagnie avec plus de vols regionaux en cours 

In [13]:
from pyspark.sql import SparkSession
import pandas as pd 
from FlightRadar24.api import FlightRadar24API
from pyspark.sql.functions import max
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

spark = SparkSession.builder.getOrCreate()

fr_api = FlightRadar24API()

dfvoyages=spark.read.csv('firstable.csv', header=True, inferSchema=True)
dfairports=spark.read.csv('airports.csv', header=True, inferSchema=True)

dfres=dfvoyages.join(dfairports,dfvoyages.originairport_icao==dfairports.icao,"inner").select("id_vol","airline_icao","originairport_icao","destinationairport_icao",dfairports.airport_zone.alias("origin_airport_zone"))
dfres=dfres.join(dfairports,dfres.destinationairport_icao==dfairports.icao,"inner")
dfres=dfres.select(dfres.id_vol,dfres.airline_icao,dfres.originairport_icao,dfres.destinationairport_icao,dfres.origin_airport_zone,dfres.airport_zone.alias("destination_airport_zone"))
#dfres.show()

dfres.createOrReplaceTempView("rslt")

df=spark.sql("select origin_airport_zone as airport_zone, airline_icao , count(distinct id_vol) from rslt where origin_airport_zone = destination_airport_zone group by airline_icao , origin_airport_zone").withColumnRenamed("count(DISTINCT id_vol)", "nbre_vol")
display(df.toPandas())

dfsec=df.groupBy('airport_zone').agg(({"nbre_vol":"max"})).withColumnRenamed("max(nbre_vol)", "nbre_vol_max")
dfsec=dfsec.join(df,(df.airport_zone==dfsec.airport_zone) & (df.nbre_vol == dfsec.nbre_vol_max),"inner").select(dfsec.airport_zone,dfsec.nbre_vol_max,"airline_icao")

display(dfsec.toPandas())
sc.stop()


,airport_zone,airline_icao,nbre_vol
0,europe,AFR,1
1,asia,QTR,1
2,northamerica,FDX,3
3,europe,PGT,1
4,northamerica,AAL,9
5,africa,ETH,4
6,northamerica,UPS,2
7,northamerica,CKS,1
8,oceania,ANZ,1
9,asia,FDX,1


,airport_zone,nbre_vol_max,airline_icao
0,europe,1,PGT
1,europe,1,AFR
2,oceania,3,QFA
3,africa,4,ETH
4,northamerica,9,AAL
5,asia,1,ETD
6,asia,1,SVA
7,asia,1,FDX
8,asia,1,QTR


In [1]:
import findspark
findspark.init()
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


Nombre de vol de chaque airline 
Airline avec plus de vols en cours 

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import max
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


df1=spark.read.csv('firstable.csv', header=True, inferSchema=True)
df1.createOrReplaceTempView("voyages")
df2=spark.read.csv('airports.csv', header=True, inferSchema=True)
df2.createOrReplaceTempView("airports")

df=spark.sql("select airline_icao , count(distinct id_vol ) as nombre_vols from my_table group by airline_icao")
df.show()
df.createOrReplaceTempView("t3")
df2=spark.sql("select airline_icao , nombre_vols from t3 order by nombre_vols asc limit 1")
df2.show()

sc.stop()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `my_table` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 66;
'Aggregate ['airline_icao], ['airline_icao, 'count(distinct 'id_vol) AS nombre_vols#232732]
+- 'UnresolvedRelation [my_table], [], false


Pour chaque continent, la compagnie avec le + de vols régionaux actifs (continent d'origine == continent de destination)


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import max

df1=spark.read.csv('firstable.csv', header=True, inferSchema=True)
df1.createOrReplaceTempView("my_table")

